In [1]:
import requests,json,datetime,time
import pandas as pd
from io import StringIO as sio
pd.options.display.float_format = '{:,.2f}'.format
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.express as px

header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.66',
    'Accept': '*/*',
    'content-type': 'application/json',
    'Accept-Encoding': 'gzip, deflate, br',
    }

def getaio(df):
    buyer_stack=df.pivot_table(index="symbol", columns='buyer',values=['qty','amt'],aggfunc={'qty':['sum','count'],'amt': 'sum'}).stack()
    buyer_stack.columns.droplevel()
    buyer_stack.columns='amount count qty'.split()
    buyer_stack.columns='buy_'+buyer_stack.columns
    seller_stack=df.pivot_table(index="symbol", columns='seller',values=['qty','amt'],aggfunc={'qty':['sum','count'],'amt': 'sum'}).stack()
    seller_stack.columns.droplevel()
    seller_stack.columns='amount count qty'.split()
    seller_stack.columns='sale_'+seller_stack.columns
    buy_sale_stack=pd.concat([buyer_stack,seller_stack],axis=1)
    buy_sale_stack.index.names=('symbol','broker')
    buy_sale_stack.fillna(0,inplace=True)
    buy_pct=buy_sale_stack.buy_qty.groupby(level=0).apply(lambda x: 100* x/x.sum())
    sale_pct=buy_sale_stack.sale_qty.groupby(level=0).apply(lambda x: 100* x/x.sum())
    aio=pd.concat([buy_sale_stack,buy_pct,sale_pct],axis=1)
    col_name=list(aio.columns)
    col_name[-2:]=["buy_pct",'sale_pct']
    aio.columns=col_name
    aio.fillna(0,inplace=True)
    return aio
def get_fs(url):
    df=pd.read_csv(sio(requests.get(url).text),thousands=',',low_memory=False)
    df=df["TransactionNumber Symbol	BuyerBrokerCode	SellerBrokerCode	Quantity	Rate	Amount".split()]
    df.columns="contract symbol buyer seller qty rate amt".split()
    df.contract=df.contract.astype(str)
    df.buyer=df.buyer.astype(str)
    df.seller=df.seller.astype(str)
    return df

def latest_n_days(df,n):
  unk=df.index.unique(level='date').sort_values(ascending=False)
  return df.loc[unk[:n].sort_values()]

chart_provider={
'nepsechart':'https://ohlcv.nepsechart.com/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}',
'merocapital':'https://chartdata.merocapital.com/datafeed1/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}',
'merolaganida':'https://da.merolagani.com/handlers/TechnicalChartHandler.ashx?type=get_advanced_chart&symbol={symbol}&resolution={resolution}&rangeStartDate={fromtime}&rangeEndDate={totime}&from=&isAdjust=1&currencyCode=NPR',
'merolagani':'https://www.merolagani.com/handlers/TechnicalChartHandler.ashx?type=get_advanced_chart&symbol={symbol}&resolution={resolution}&rangeStartDate={fromtime}&rangeEndDate={totime}&from=&isAdjust=1&currencyCode=NPR',
'nepsealpha':'https://nepsealpha.com/trading/1/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}&currencyCode=NRS',
'nepsedata':'https://nepsedata.com/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}&currencyCode=NRS'
}
def chart_data(provider='merolagani',symbol="NEPSE",fromtime=datetime.date(2019,1,1),totime=datetime.datetime.now(),resolution="1D"):
    """returns data fetched from nepsealpha,merocapital,merolagani and nepsechart.\nwarning!! \nNepsechart,merocapital takes D for daily resolution.\nMerocapital,nepsedata gives unadjusted chart)"""
    fromtime =int(time.mktime(fromtime.timetuple()))
    totime =int(time.mktime(totime.timetuple())) 
    url=chart_provider.get(provider)
    df=pd.read_json(requests.get(url.format(symbol=symbol,resolution=resolution,fromtime=fromtime,totime=totime),headers=header).content)
    df['t']=df['t'].apply(datetime.datetime.utcfromtimestamp)
    df.drop('s',axis=1,inplace=True)
    return df

In [2]:
def get_madhuko_fs(dater):
  url="https://raw.githubusercontent.com/madhuko/temp/main/fs/{}".format(dater)
  df=pd.read_csv(url,low_memory=False)
  df.contract=df.contract.astype(str)
  df.buyer=df.buyer.astype(str)
  df.seller=df.seller.astype(str)
  df.qty=df.qty.astype(float)
  df.rate=df.rate.astype(float)
  df.amt=df.amt.astype(float)
  contract_1=df.contract[1]
  if dater!=datetime.date(int(contract_1[:4]),int(contract_1[4:6]),int(contract_1[6:8])):
    print("date valdation failed for {}".format(dater))
  return df
  

def getfs_nepsealpha(symbol):
  fs=s.get("https://nepsealpha.com/floorsheet_ajx/{}/index".format(symbol))
  new_df=pd.read_html(sio(fs.json()['html']))
  new_df[1]["Symbol"]=symbol
  return new_df[1]

def get_latest_fs():
    s=requests.Session()
    s.headers.update(header)
    s.get("https://nepsealpha.com/trading/chart")
    df=pd.DataFrame()
    live_market=pd.read_html("https://www.merolagani.com/LatestMarket.aspx")[0]
    for sym in live_market["Symbol"]:
        if "/" in sym:
            continue
        ram=getfs_nepsealpha(sym)
        df=pd.concat([df,ram])
        print("Collected data of {}".format(sym))
    df.columns='contract buyer seller qty rate amt symbol'.split()
    df["amt"]=df["amt"].apply(lambda x: x.replace("NPR",""))
    df["amt"]=df["amt"].apply(lambda x: x.replace(",",""))
    df.amt=df.amt.astype(float)
    df.buyer=df.buyer.astype(str)
    df.seller=df.seller.astype(str)
    df["rate"]=df["rate"].apply(lambda x: x.replace("NPR ",""))
    return df

In [5]:
init_ohlc=chart_data(fromtime=datetime.date(2021,11,1))
init_ohlc.t=init_ohlc.t.dt.date
init_ohlc.sort_values('t',ascending=False,inplace=True)
aio=pd.DataFrame()

# get d1 data
dater=init_ohlc.t[0]
try:
    df=get_madhuko_fs(dater)
except:
    df=get_latest_fs()
taio=getaio(df)
taio['date']=dater
taio=taio.reset_index().set_index(['date','symbol','broker'])
aio=pd.concat([aio,taio])
print(dater)

for dater in init_ohlc.t[1:]:
  df=get_madhuko_fs(dater)
  taio=getaio(df)
  taio['date']=dater
  taio=taio.reset_index().set_index(['date','symbol','broker'])
  aio=pd.concat([aio,taio])
  print(dater)

2021-10-31
2021-11-08
2021-11-03
2021-11-02
2021-11-01
2021-10-31


In [20]:
abc=init_ohlc.sort_values('o',ascending=False).copy()

In [33]:
init_ohlc.sort_index(inplace=True)

In [36]:
init_ohlc.sort_values(by='v',ascending=False)

,t,o,h,l,c,v
0,2021-10-31,"2,785.95","2,838.09","2,775.54","2,837.62",7765960000
1,2021-11-01,"2,852.32","2,866.44","2,804.31","2,819.87",7690020000
3,2021-11-03,"2,858.74","2,878.20","2,837.59","2,848.84",6510540000
5,2021-11-09,"2,831.00","2,833.05","2,730.21","2,749.86",6507040000
2,2021-11-02,"2,824.83","2,845.89","2,789.20","2,843.62",5985940000
4,2021-11-08,"2,854.38","2,863.80","2,821.64","2,829.65",4398550000
